<a href="https://colab.research.google.com/github/MariiaZimokha/notebooks/blob/main/kaggle/Fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set up

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = "/content/drive/MyDrive/Studying/PS_20174392719_1491204439457_log.csv"

# Imports




In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from zipfile import ZipFile

import os.path

In [ ]:
df = pd.read_csv(file_path)

In [ ]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


The data is coming from https://www.kaggle.com/datasets/ealaxi/paysim1?resource=download

1. **step**: represents a unit of time where 1 step equals 1 hour
2. **type**: type of online transaction
3. **amount**: the amount of the transaction
4. **nameOrig**: customer starting the transaction
5. **oldbalanceOrg**: balance before the transaction
6. **newbalanceOrig**: balance after the transaction
7. **nameDest**: recipient of the transaction
8. **oldbalanceDest**: initial balance of recipient before the transaction
9. **newbalanceDest**: the new balance of recipient after the transaction
10. isFraud: fraud transaction

In [ ]:
print(df.isnull().sum())

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64


In [ ]:
df["type"].unique()

array(['PAYMENT', 'TRANSFER', 'CASH_OUT', 'DEBIT', 'CASH_IN'],
      dtype=object)

In [ ]:
df["type"].value_counts()

CASH_OUT    2237500
PAYMENT     2151495
CASH_IN     1399284
TRANSFER     532909
DEBIT         41432
Name: type, dtype: int64

In [ ]:
type = df["type"].value_counts()
transactions = type.index
quantity = type.values

import plotly.express as px

figure = px.pie(df, 
             values=quantity, 
             names=transactions,
             hole = 0.5, 
             title="Distribution of Transaction Type")
figure.show()

In [ ]:
category_to_int = {
    'PAYMENT': 1, 
    'TRANSFER': 2, 
    'CASH_OUT': 3, 
    'DEBIT': 4, 
    'CASH_IN': 5
}

In [ ]:
df["type"] = df["type"].map(category_to_int)

In [ ]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,1,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,1,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,2,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,3,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,1,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [ ]:
correlation = df.corr()
correlation["isFraud"].sort_values(ascending=False)

<ipython-input-13-00580b5e44f6>:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



isFraud           1.000000
amount            0.076688
isFlaggedFraud    0.044109
step              0.031578
oldbalanceOrg     0.010154
newbalanceDest    0.000535
type             -0.004463
oldbalanceDest   -0.005885
newbalanceOrig   -0.008148
Name: isFraud, dtype: float64

**When the value is close to 1, it means that there is a strong positive correlation, and when the value is close to -1, it means that there is a strong negative correlation. And when the values are close to 0, it means that there is no correlation.**

### Modeling

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth=3)

In [ ]:
# splitting the data
from sklearn.model_selection import train_test_split
X = np.array(df[["amount", "isFlaggedFraud", "newbalanceOrig", "oldbalanceDest"]])
Y = np.array(df[["isFraud"]])

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)
# x = np.array(data[["type", "amount", "oldbalanceOrg", "newbalanceOrig"]])
# y = np.array(data[["isFraud"]])

In [ ]:
model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=3)

In [ ]:
model.score(X_test, y_test)

0.998762040375401

In [ ]:
from sklearn import tree
tree.plot_tree(model)